I am trying to test reinjection for different structures. this is just a general test.

the idea is to remove the acsl in combined file in a new column, and then reinject right after.

combined_file -> code_implementation -> combined_file

In [2]:
import pandas as pd
import re
import ast
import modal
from validation_treesitter import inject_acsl_specs_treesitter, extract_structure, validate_and_inject

In [6]:
df = pd.read_csv('../hidden/rlfr_hidden.csv')
df = df[['prompt', 'spec', 'combined_file']]
df.head()

,prompt,spec,combined_file
0,"Given three integers `v`, `min`, and `max`, re...","[[], '/*@ assigns \\nothing;\n ensures \\re...",/*@ assigns \nothing;\n ensures \result == ...
1,## Problem Description\n\nGiven an integer arr...,"[[], '/*@ requires n > 0;\n assigns \\nothi...",/*@ requires n > 0;\n assigns \nothing;\n ...
2,"## Problem Description\n\nGiven a string, dete...","[[""/*@ predicate matcher_ab(char * x0) = ((((...",#include <limits.h>\n#include <string.h>\n/*@ ...
3,## Problem Description\n\nGiven an integer arr...,"[[], '/*@\n\trequires n>0;\n\trequires 0<=k<n;...",/*@\n\trequires n>0;\n\trequires 0<=k<n;\n\tre...
4,## Problem Description\n\nGiven an integer arr...,"[[], '/*@\n\trequires n>0;\n\trequires\\valid(...",/*@\n\trequires n>0;\n\trequires\valid(a+ (0.....


In [7]:
def remove_all_comments(c_code: str) -> str:
    if pd.isna(c_code) or not c_code:
        return ""
    c_code = re.sub(r'/\*.*?\*/', '', c_code, flags=re.DOTALL)
    c_code = re.sub(r'//.*?$', '', c_code, flags=re.MULTILINE)
    c_code = re.sub(r'\n\s*\n+', '\n\n', c_code)
    
    return c_code.strip()

df['code_implementation'] = df['combined_file'].apply(remove_all_comments)
df.head()

,prompt,spec,combined_file,code_implementation
0,"Given three integers `v`, `min`, and `max`, re...","[[], '/*@ assigns \\nothing;\n ensures \\re...",/*@ assigns \nothing;\n ensures \result == ...,"int clamp(int v, int min, int max)\n{\n\tint l..."
1,## Problem Description\n\nGiven an integer arr...,"[[], '/*@ requires n > 0;\n assigns \\nothi...",/*@ requires n > 0;\n assigns \nothing;\n ...,int pick_index(int n) { return 0; }\n\nint pic...
2,"## Problem Description\n\nGiven a string, dete...","[[""/*@ predicate matcher_ab(char * x0) = ((((...",#include <limits.h>\n#include <string.h>\n/*@ ...,#include <limits.h>\n#include <string.h>\n\nin...
3,## Problem Description\n\nGiven an integer arr...,"[[], '/*@\n\trequires n>0;\n\trequires 0<=k<n;...",/*@\n\trequires n>0;\n\trequires 0<=k<n;\n\tre...,"void q2(int a[],int n,int k)\n{\n\n\tfor (int ..."
4,## Problem Description\n\nGiven an integer arr...,"[[], '/*@\n\trequires n>0;\n\trequires\\valid(...",/*@\n\trequires n>0;\n\trequires\valid(a+ (0.....,"void bubbleinnerloop(int a[], int n)\n{\n\n\tf..."


In [8]:
row = df.iloc[0]
code_impl = row['code_implementation']

print('Initial:\n' + row['combined_file'] + '\n')
print('Reinjected:\n' + inject_acsl_specs_treesitter(ast.literal_eval(row['spec']), code_impl) + '\n')

Initial:
/*@ assigns \nothing;
    ensures \result == \min(\max(min, v), max);
 */
int clamp(int v, int min, int max)
{
	int low = v > min ? v : min;
	return low < max ? low : max;
}

Reinjected:
/*@ assigns \nothing;
    ensures \result == \min(\max(min, v), max);
 */
int clamp(int v, int min, int max)
{
	int low = v > min ? v : min;
	return low < max ? low : max;
}



In [9]:
def reinject_row(row):
    try:
        spec = ast.literal_eval(row['spec'])
        return inject_acsl_specs_treesitter(spec, row['code_implementation'])
    except Exception as e:
        return f"ERROR: {e}"

df['reinjection'] = df.apply(reinject_row, axis=1)
df.head()

,prompt,spec,combined_file,code_implementation,reinjection
0,"Given three integers `v`, `min`, and `max`, re...","[[], '/*@ assigns \\nothing;\n ensures \\re...",/*@ assigns \nothing;\n ensures \result == ...,"int clamp(int v, int min, int max)\n{\n\tint l...",/*@ assigns \nothing;\n ensures \result == ...
1,## Problem Description\n\nGiven an integer arr...,"[[], '/*@ requires n > 0;\n assigns \\nothi...",/*@ requires n > 0;\n assigns \nothing;\n ...,int pick_index(int n) { return 0; }\n\nint pic...,/*@ requires n > 0;\n assigns \nothing;\n ...
2,"## Problem Description\n\nGiven a string, dete...","[[""/*@ predicate matcher_ab(char * x0) = ((((...",#include <limits.h>\n#include <string.h>\n/*@ ...,#include <limits.h>\n#include <string.h>\n\nin...,/*@ predicate matcher_ab(char * x0) = ((((x0[...
3,## Problem Description\n\nGiven an integer arr...,"[[], '/*@\n\trequires n>0;\n\trequires 0<=k<n;...",/*@\n\trequires n>0;\n\trequires 0<=k<n;\n\tre...,"void q2(int a[],int n,int k)\n{\n\n\tfor (int ...",/*@\n\trequires n>0;\n\trequires 0<=k<n;\n\tre...
4,## Problem Description\n\nGiven an integer arr...,"[[], '/*@\n\trequires n>0;\n\trequires\\valid(...",/*@\n\trequires n>0;\n\trequires\valid(a+ (0.....,"void bubbleinnerloop(int a[], int n)\n{\n\n\tf...",/*@\n\trequires n>0;\n\trequires\valid(a+ (0.....


In [ ]:
df.drop(columns=['code_implementation','spec'], inplace=True)

In [ ]:
df.to_csv('compare.csv', index=False)

In [11]:
# Test reinjection with Frama-C verification
verify_annotated_c = modal.Function.from_name("frama_check", "verify_annotated_c")

# Spawn all Frama-C verification jobs in parallel
reinjection_codes = df['reinjection'].tolist()
print(f"Spawning {len(reinjection_codes)} Frama-C verification checks...")

handles = [verify_annotated_c.spawn(code) for code in reinjection_codes]
print(f"All jobs spawned. Collecting results...")

# Collect results
frama_results = [h.get() for h in handles]

# Add results as new column
df['frama_verified'] = frama_results

# Summary
print(f"\nTotal: {len(df)}")
print(f"Verified: {sum(frama_results)}")
print(f"Not verified: {len(frama_results) - sum(frama_results)}")
print(f"Pass rate: {sum(frama_results)/len(frama_results)*100:.1f}%")

# Show failures
frama_failures = df[~df['frama_verified']]
print(f"\nFailed rows: {len(frama_failures)}")
if len(frama_failures) > 0:
    print(frama_failures.index.tolist()[:20])

Spawning 346 Frama-C verification checks...
All jobs spawned. Collecting results...

Total: 346
Verified: 323
Not verified: 23
Pass rate: 93.4%

Failed rows: 23
[20, 26, 33, 49, 59, 69, 77, 79, 92, 99, 114, 130, 135, 155, 158, 160, 250, 251, 291, 293]


#### Issues:
When we ran reinjection around 23 failed.

In [13]:
# Check specific failed rows with Frama-C verification
rows_to_check = [20, 26, 33, 49, 59, 69, 77, 79, 92, 99, 114, 130, 135, 155, 158, 160, 250, 251, 291, 293]

# Spawn all Frama-C verification jobs in parallel
reinjection_codes = [df.iloc[idx]['reinjection'] for idx in rows_to_check]
print(f"Spawning {len(reinjection_codes)} Frama-C verification checks...")

handles = [verify_annotated_c.spawn(code) for code in reinjection_codes]
print(f"All jobs spawned. Collecting results...")

# Collect results
frama_results = [h.get() for h in handles]

# Summary
print(f"\nTotal: {len(rows_to_check)}")
print(f"Verified: {sum(frama_results)}")
print(f"Not verified: {len(frama_results) - sum(frama_results)}")
print(f"Pass rate: {sum(frama_results)/len(frama_results)*100:.1f}%")

# Show failures
for idx, result in zip(rows_to_check, frama_results):
    if not result:
        print(f"\nRow {idx} failed:")

Spawning 20 Frama-C verification checks...
All jobs spawned. Collecting results...

Total: 20
Verified: 0
Not verified: 20
Pass rate: 0.0%

Row 20 failed:

Row 26 failed:

Row 33 failed:

Row 49 failed:

Row 59 failed:

Row 69 failed:

Row 77 failed:

Row 79 failed:

Row 92 failed:

Row 99 failed:

Row 114 failed:

Row 130 failed:

Row 135 failed:

Row 155 failed:

Row 158 failed:

Row 160 failed:

Row 250 failed:

Row 251 failed:

Row 291 failed:

Row 293 failed:


In [ ]:
# Create new df with only the failed rows
failed_rows = [20, 26, 33, 49, 59, 69, 77, 79, 92, 99, 114, 130, 135, 155, 158, 160, 250, 251, 291, 293]
df_failed = df.iloc[failed_rows].copy()
df_failed.head()


In [16]:
df_failed = df_failed[['combined_file', 'reinjection']]

In [17]:
df_failed.head()

,combined_file,reinjection
20,/*@\n requires n >= 0;\n requires \valid...,/*@\n requires n >= 0;\n requires \valid...
26,#include<string.h>\n\n/*@\npredicate matchhere...,"/*@\npredicate matchhere(char* r, char* s) =\n..."
33,#include <limits.h>\n/*@ predicate inv_vec_Int...,"/*@ predicate inv_vec_Int(int * x0, int x1) ..."
49,/*@ terminates \true;\n @ assigns \nothing;\n...,void exit(int status);\n\nint status;\n\n/*@ t...
59,/*@ requires a_valid: \valid(a);\n requires...,"void swap(int* a, int *b);\n\n/*@ requires a_v..."


In [18]:
df_failed.to_csv('failed_rows.csv', index=False)